In [10]:
import pandas as pd
import pickle
country = {}
a_file = open("airport.pkl", "rb")
country = pickle.load(a_file)
a_file.close()
continents = {}
a_file = open("continents.pkl", "rb")
continents = pickle.load(a_file)
a_file.close()

In [11]:
sirena = pd.read_csv('data/Sirena-export-fixed.csv', index_col = 'id')

In [12]:
sirena

,PaxName,BirthDate,DepartDate,DepartTime,ArrivalDate,ArrivalTime,FlightCode,From,Dest,Code e-Ticket,TravelDoc,Seat,Meal,TravelClass,PaxAdditionalInfo,Loyalty,AgentInfo,Fare,Baggage
id,,,,,,,,,,,,,,,,,,,
0,ОЗЕРОВ ИЛЬДАР ДАНИИЛОВИЧ,1999-05-15,2017-05-30,00:05,2017-05-30,08:05,SU1306NO,SVO,OVB,ZBQSPY7360415302044672,9375 053270,NaN,NaN,J,S,FF#SU 38116280,Go2See,JGRPGN,0PC
1,КОЛОСОВ САМИР ТАМЕРЛАНОВИЧ,NaN,2017-12-27,02:15,2017-12-27,04:40,SU1323NO,MMK,SVO,MHPBBX7398421117936516,2244 645520,NaN,KSML,Y,9,FF#FB 284903754,NaN,YRSTUQ,NaN
2,ИГНАТОВА СНЕЖАНА КОНСТАНТИНОВНА,NaN,2017-09-19,06:40,2017-09-19,07:45,SU1481NO,KJA,SVO,REDLVB5174973140468001,8115 961316,NaN,NaN,Y,F,NaN,KupiBilet,YSTNJL,NaN
3,ЖАРОВ ПЛАТОН АЛЬБЕРТОВИЧ,1999-05-02,2017-03-18,22:10,2017-03-19,01:05,SU1180NO,SVO,VOG,NSJNGQ5274206497242737,98 6865148,NaN,NaN,J,#,FF#FB 884556993,Travelgenio,JFLXLS,0PC
4,НИКОЛЬСКИЙ НИКОЛАЙ ИГОРЕВИЧ,1990-12-26,2017-03-18,22:10,2017-03-19,01:05,SU1180NO,SVO,VOG,VWNYGF6247422701565929,4396 926588,NaN,NaN,Y,NaN,FF#SU 183142068,OZON.travel,YFLXPG,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155752,ЕМЕЛЬЯНОВ МИРОСЛАВ СВЯТОСЛАВОВИЧ,NaN,2017-02-14,03:10,2017-02-14,04:00,SU1181NO,VOG,SVO,JYYXED4714800502596692,3511 319326,NaN,STML,Y,F,NaN,OneTwoTrip,YSTNER,NaN
155753,ТАРАСОВ АРТЕМИЙ ГЕРМАНОВИЧ,NaN,2017-02-14,03:10,2017-02-14,04:00,SU1181NO,VOG,SVO,XDKDZF6730365623280343,2199 078129,NaN,NaN,Y,T,FF#DT 261604825,OZON.travel,YFLXKV,1PC
155754,ГОРОХОВ БОРИС СВЯТОГОРОВИЧ,1970-10-28,2017-02-14,03:10,2017-02-14,04:00,SU1181NO,VOG,SVO,SOWGLC1569746161308219,7351 760618,NaN,NaN,Y,F,FF#FB 902862806,Aeroflot,YFLXBF,NaN


### Для перевода имени колонок к удобному виду

In [13]:
import re
def camel_to_snake(name):
  name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
  return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()

In [14]:
new_cols = {i:camel_to_snake(i) for i in sirena.columns}
sirena.rename(columns=new_cols, inplace=True)

### Считаем все: not_ok, между народные, ненормальные международные и кроссконтинентальные

In [15]:
def count_not_normal_flights(dataframe):
    prev_air = ''
    not_ok = 0
    cross_country = 0
    cross_not_ok = 0
    long = 0
    for elem in dataframe.values:
        from_air = elem[0]
        to_air = elem[1]
        from_coun = elem[2]
        to_coun = elem[3]
        from_con = elem[4]
        to_con = elem[5]
        if pd.isna(from_air):
            continue
        if pd.isna(to_air):
            prev_air = ''
            continue
        if prev_air != '' and from_air.lower() != prev_air.lower():
            not_ok += 1
            if from_coun != '' and to_coun != '':
                if from_coun.lower() != to_coun.lower():
                    cross_not_ok += 1
        prev_air = to_air
        if from_coun != '' and to_coun != '':
            if from_coun.lower() != to_coun.lower():
                cross_country += 1
        if from_con != '' and to_con != '':
            if from_con.lower() != to_con.lower():
                long += 1
    return not_ok, cross_country, cross_not_ok, long

In [16]:
sirena

,pax_name,birth_date,depart_date,depart_time,arrival_date,arrival_time,flight_code,from,dest,code e-_ticket,travel_doc,seat,meal,travel_class,pax_additional_info,loyalty,agent_info,fare,baggage
id,,,,,,,,,,,,,,,,,,,
0,ОЗЕРОВ ИЛЬДАР ДАНИИЛОВИЧ,1999-05-15,2017-05-30,00:05,2017-05-30,08:05,SU1306NO,SVO,OVB,ZBQSPY7360415302044672,9375 053270,NaN,NaN,J,S,FF#SU 38116280,Go2See,JGRPGN,0PC
1,КОЛОСОВ САМИР ТАМЕРЛАНОВИЧ,NaN,2017-12-27,02:15,2017-12-27,04:40,SU1323NO,MMK,SVO,MHPBBX7398421117936516,2244 645520,NaN,KSML,Y,9,FF#FB 284903754,NaN,YRSTUQ,NaN
2,ИГНАТОВА СНЕЖАНА КОНСТАНТИНОВНА,NaN,2017-09-19,06:40,2017-09-19,07:45,SU1481NO,KJA,SVO,REDLVB5174973140468001,8115 961316,NaN,NaN,Y,F,NaN,KupiBilet,YSTNJL,NaN
3,ЖАРОВ ПЛАТОН АЛЬБЕРТОВИЧ,1999-05-02,2017-03-18,22:10,2017-03-19,01:05,SU1180NO,SVO,VOG,NSJNGQ5274206497242737,98 6865148,NaN,NaN,J,#,FF#FB 884556993,Travelgenio,JFLXLS,0PC
4,НИКОЛЬСКИЙ НИКОЛАЙ ИГОРЕВИЧ,1990-12-26,2017-03-18,22:10,2017-03-19,01:05,SU1180NO,SVO,VOG,VWNYGF6247422701565929,4396 926588,NaN,NaN,Y,NaN,FF#SU 183142068,OZON.travel,YFLXPG,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155752,ЕМЕЛЬЯНОВ МИРОСЛАВ СВЯТОСЛАВОВИЧ,NaN,2017-02-14,03:10,2017-02-14,04:00,SU1181NO,VOG,SVO,JYYXED4714800502596692,3511 319326,NaN,STML,Y,F,NaN,OneTwoTrip,YSTNER,NaN
155753,ТАРАСОВ АРТЕМИЙ ГЕРМАНОВИЧ,NaN,2017-02-14,03:10,2017-02-14,04:00,SU1181NO,VOG,SVO,XDKDZF6730365623280343,2199 078129,NaN,NaN,Y,T,FF#DT 261604825,OZON.travel,YFLXKV,1PC
155754,ГОРОХОВ БОРИС СВЯТОГОРОВИЧ,1970-10-28,2017-02-14,03:10,2017-02-14,04:00,SU1181NO,VOG,SVO,SOWGLC1569746161308219,7351 760618,NaN,NaN,Y,F,FF#FB 902862806,Aeroflot,YFLXBF,NaN


### Определяем страны прилета и вылета по коду аэропорта

In [17]:
sirena['dep_country'] = [country.get(elem,'') for elem in sirena['from'].values]

In [18]:
sirena['arr_country'] = [country.get(elem,'') for elem in sirena['dest'].values]

In [19]:
sirena['dep_continent'] = [continents.get(elem,'') for elem in sirena['dep_country'].values]

In [20]:
sirena['arr_continent'] = [continents.get(elem,'') for elem in sirena['arr_country'].values]

In [21]:
sirena

,pax_name,birth_date,depart_date,depart_time,arrival_date,arrival_time,flight_code,from,dest,code e-_ticket,...,travel_class,pax_additional_info,loyalty,agent_info,fare,baggage,dep_country,arr_country,dep_continent,arr_continent
id,,,,,,,,,,,,,,,,,,,,,
0,ОЗЕРОВ ИЛЬДАР ДАНИИЛОВИЧ,1999-05-15,2017-05-30,00:05,2017-05-30,08:05,SU1306NO,SVO,OVB,ZBQSPY7360415302044672,...,J,S,FF#SU 38116280,Go2See,JGRPGN,0PC,Russia,Russia,Europe,Europe
1,КОЛОСОВ САМИР ТАМЕРЛАНОВИЧ,NaN,2017-12-27,02:15,2017-12-27,04:40,SU1323NO,MMK,SVO,MHPBBX7398421117936516,...,Y,9,FF#FB 284903754,NaN,YRSTUQ,NaN,Russia,Russia,Europe,Europe
2,ИГНАТОВА СНЕЖАНА КОНСТАНТИНОВНА,NaN,2017-09-19,06:40,2017-09-19,07:45,SU1481NO,KJA,SVO,REDLVB5174973140468001,...,Y,F,NaN,KupiBilet,YSTNJL,NaN,Russia,Russia,Europe,Europe
3,ЖАРОВ ПЛАТОН АЛЬБЕРТОВИЧ,1999-05-02,2017-03-18,22:10,2017-03-19,01:05,SU1180NO,SVO,VOG,NSJNGQ5274206497242737,...,J,#,FF#FB 884556993,Travelgenio,JFLXLS,0PC,Russia,Russia,Europe,Europe
4,НИКОЛЬСКИЙ НИКОЛАЙ ИГОРЕВИЧ,1990-12-26,2017-03-18,22:10,2017-03-19,01:05,SU1180NO,SVO,VOG,VWNYGF6247422701565929,...,Y,NaN,FF#SU 183142068,OZON.travel,YFLXPG,NaN,Russia,Russia,Europe,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155752,ЕМЕЛЬЯНОВ МИРОСЛАВ СВЯТОСЛАВОВИЧ,NaN,2017-02-14,03:10,2017-02-14,04:00,SU1181NO,VOG,SVO,JYYXED4714800502596692,...,Y,F,NaN,OneTwoTrip,YSTNER,NaN,Russia,Russia,Europe,Europe
155753,ТАРАСОВ АРТЕМИЙ ГЕРМАНОВИЧ,NaN,2017-02-14,03:10,2017-02-14,04:00,SU1181NO,VOG,SVO,XDKDZF6730365623280343,...,Y,T,FF#DT 261604825,OZON.travel,YFLXKV,1PC,Russia,Russia,Europe,Europe
155754,ГОРОХОВ БОРИС СВЯТОГОРОВИЧ,1970-10-28,2017-02-14,03:10,2017-02-14,04:00,SU1181NO,VOG,SVO,SOWGLC1569746161308219,...,Y,F,FF#FB 902862806,Aeroflot,YFLXBF,NaN,Russia,Russia,Europe,Europe


In [22]:
len(sirena.pax_name.unique())

72735

### Создаем таблицу для записи данных

In [29]:
not_normal_flights = pd.DataFrame(columns=['name', 'flights', 'not_ok_flights', 'cross_country', 'cross_country_not_ok', 'long_flights'])

In [30]:
value = 5000
unique_names = sirena.pax_name.unique()
for i in range (len(unique_names)):
    name = unique_names[i]
    sub_df = sirena[sirena['pax_name'] == name].sort_values(by='depart_date').reset_index(drop=True)
    sub_df_1 = sub_df[['from', 'dest', 'dep_country', 'arr_country', 'dep_continent', 'arr_continent']]
    count_flights = len(sub_df)
    not_ok, cross_country, cross_not_ok, long = count_not_normal_flights(sub_df_1)
    not_normal_flights.loc[i] = [name] + [count_flights] + [not_ok] + [cross_country] + [cross_not_ok] + [long]
    if i == value:
        print(i)
        value += 5000

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000


In [31]:
not_normal_flights

,name,flights,not_ok_flights,cross_country,cross_country_not_ok,long_flights
0,ОЗЕРОВ ИЛЬДАР ДАНИИЛОВИЧ,1,0,0,0,0
1,КОЛОСОВ САМИР ТАМЕРЛАНОВИЧ,7,3,0,0,0
2,ИГНАТОВА СНЕЖАНА КОНСТАНТИНОВНА,5,1,0,0,0
3,ЖАРОВ ПЛАТОН АЛЬБЕРТОВИЧ,5,0,0,0,0
4,НИКОЛЬСКИЙ НИКОЛАЙ ИГОРЕВИЧ,6,2,0,0,0
...,...,...,...,...,...,...
72730,МАСЛОВА ЯСМИНА МАКАРЬЕВНА,1,0,0,0,0
72731,БОЧАРОВА МИЛЕНА ДЕМИДОВНА,1,0,0,0,0
72732,ЯКОВЛЕВ АРТЁМ АНАТОЛЬЕВИЧ,1,0,0,0,0
72733,ОЗЕРОВ ГЕОРГИЙ СЕРГЕЕВИЧ,1,0,0,0,0


### Вычисляем процент подозрительных рейсов и процент международных рейсов

In [32]:
not_normal_flights['percent_not_ok'] = [elem[2] / elem[1] for elem in not_normal_flights.values]

In [33]:
not_normal_flights['percent_cross'] = [elem[3] / elem[1] for elem in not_normal_flights.values]

In [34]:
not_normal_flights.to_csv('data/Ready/not_norm_sirena_1.csv', index_label='id')